# Lesson 1 : Introduction
  Python Serial and Arduino Serial

# Lesson 2 : Installation
1. Python
2. Python Library : pySerial
3. vPython -> For visualize the data

In [ ]:
# Lesson 2 : Example Code
# .ino code
int cnt=0;
void setup(){
    Serial.Begin(9600);
}
void loop(){
    Serial.print("Im counting :");
    Serial.print(cnt);
    Serial.println("Missisipi");
    cnt = cnt + 1;
    delay(1000);
}

In [ ]:
# .py code
import serial

arduinoserial = serial.Serial('COM11',)

while True:
  if(arduinoserial.inWaiting()>0):
    mydata = arduinoserial.readline()  # it will read into string so deal with it later
    print(mydata)

# Lesson 3 : Create a Simple Virtual World
Getting data from Ultrasonic sensor in arduino for read and visualize using python

In [ ]:
# .ino code
int trigPin=13; //Sensor Trig pin connected to Arduino pin 13
int echoPin=11;  //Sensor Echo pin connected to Arduino pin 11
float pingTime;  //time for ping to travel from sensor to target and return
float targetDistance; //Distance to Target in inches
float speedOfSound=776.5; //Speed of sound in miles per hour when temp is 77 degrees.
 
void setup() {
  // put your setup code here, to run once:
  Serial.begin(9600);
  pinMode(trigPin, OUTPUT);
  pinMode(echoPin, INPUT);
}
 
void loop() {
  // put your main code here, to run repeatedly:
  
  digitalWrite(trigPin, LOW); //Set trigger pin low
  delayMicroseconds(2000); //Let signal settle
  digitalWrite(trigPin, HIGH); //Set trigPin high
  delayMicroseconds(15); //Delay in high state
  digitalWrite(trigPin, LOW); //ping has now been sent
  delayMicroseconds(10); //Delay in low state
  
  pingTime = pulseIn(echoPin, HIGH);  //pingTime is presented in microceconds
  pingTime=pingTime/1000000; //convert pingTime to seconds by dividing by 1000000 (microseconds in a second)
  pingTime=pingTime/3600; //convert pingtime to hourse by dividing by 3600 (seconds in an hour)
  targetDistance= speedOfSound * pingTime;  //This will be in miles, since speed of sound was miles per hour
  targetDistance=targetDistance/2; //Remember ping travels to target and back from target, so you must divide by 2 for actual target distance.
  targetDistance= targetDistance*63360;    //Convert miles to inches by multipling by 63360 (inches per mile)
  
  Serial.println(targetDistance);
  
  delay(100); //delay tenth of a  second to slow things down a little.
}

In [ ]:
# .py code
import serial 
from visual import * #Import all the vPython library
 
arduinoSerialData = serial.Serial('com11', 9600) 
measuringRod = cylinder( radius= .1, length=6, color=color.yellow, pos=(-3,-2,0))
lengthLabel = label(pos=(0,5,0), text='Target Distance is: ', box=false, height=30)
target=box(pos=(0,-.5,0), length=.2, width=3, height=3, color=color.green)
while (1==1):
    rate(20)#Tell vpython to run this loop 20 times a second
    if (arduinoSerialData.inWaiting()>0):  
        myData = arduinoSerialData.readline() 
        print(myData)
        distance = float(myData) #convert reading to a floating point number
        measuringRod.length=distance #Change the length of your measuring rod to your last measurement
        target.pos=(-3+distance,-.5,0)
        myLabel= 'Target Distance is: ' + myData #Create label by appending string myData to string
        lengthLabel.text = myLabel #display updated myLabel on your graphic

# Lesson 4 : Expanding Visual World
Detecting the color to change the python color and Measuring the distance using ultrasonic sensor
# Lesson 5 : vPython Library to Create a Virtual World
More Example of vpython

In [ ]:
# .ino code
int S2= 7; //Color sensore pin S2 to Arduino pin 7
int S3= 8; //Color sensor pin S3 to Arduino pin 8
int outPin = 4; //Color Sensor OUT to Arduino pin 4
 
int trigPin=13; //Ultrasonic Sensor Trig pin connected to Arduino pin 13
int echoPin=11;  //Ultrasonic Sensor Echo pin connected to Arduino pin 11
 
int rColorStrength; //measured strength of red color
int gColorStrength; //measured strength of green color
int bColorStrength; //measured strength of blue color
unsigned int pulseWidth; //for measuring color strength using pulseIn command
 
float pingTime;  //time for ping to travel from sensor to target and return
float targetDistance; //Distance to Target in inches
float speedOfSound=776.5; //Speed of sound in miles per hour when temp is 77 degrees.
 
 
void setup() {
  // put your setup code here, to run once:
  
  Serial.begin(115200); //turn on serial port
  
  pinMode(S2, OUTPUT); //S2 and S3 are outputs and used to tell
  pinMode(S3, OUTPUT); //arduino which color to measure
  pinMode(outPin, INPUT); //This is the pin we read the color from
  
  pinMode(trigPin, OUTPUT); //Ultrasonic Trig Pin is an output
  pinMode(echoPin, INPUT);  //Ultradoinic Echo Pin is an input
  
}
 
void loop() {
  
  //Lets start by reading Red Component of the Color
  // S2 and S3 should be set LOW
  digitalWrite(S2, LOW);
  digitalWrite(S3, LOW);
  
  pulseWidth =  pulseIn(outPin, LOW); //Measure raw pulsewidth coming from color sensor outpin
  
  rColorStrength = pulseWidth/400. -1; //normalize number to number between 0 and 255
  
  rColorStrength = (255- rColorStrength); //reverse so that large number means strong color
  
  
  //Lets read Green Component of the Color
  // S2 and S3 should be set HIGH
  digitalWrite(S2, HIGH);
  digitalWrite(S3, HIGH);
  
  pulseWidth =  pulseIn(outPin, LOW); //Measure raw pulsewidth coming from color sensor outpin
  
  gColorStrength = pulseWidth/400. -1; //normalize number to number between 0 and 255
  
  gColorStrength = (255- gColorStrength); //reverse so that large number means strong color
  
  gColorStrength=gColorStrength + 2;
  
  //Lets read Blue Component of the Color
  // S2 and S3 should be set LOW and HIGH Respectively
  digitalWrite(S2, LOW);
  digitalWrite(S3, HIGH);
  
  pulseWidth =  pulseIn(outPin, LOW); //Measure raw pulsewidth coming from color sensor outpin
  
  bColorStrength = pulseWidth/400. -1; //normalize number to number between 0 and 255
  
  bColorStrength = (255- bColorStrength); //reverse so that large number means strong color
  
  //Now we need to exagerate the colors because readings from color sensor 
  //are all too close together. Algorithm that appears to work is to
  //take the strongest color and set to 255, take the weakest color
  //and set to zero, and then take the middle color and reduce its
  //value by 2. That is what this next segment of code does.
 
 if(rColorStrength>gColorStrength && gColorStrength>bColorStrength) {
   
   rColorStrength = 255;
   gColorStrength = gColorStrength/2;
   bColorStrength = 0;
 }
 
  if(rColorStrength>bColorStrength && bColorStrength>gColorStrength) {
   
   rColorStrength = 255;
   bColorStrength = bColorStrength/2;
   gColorStrength = 0;
 }
 
  if(gColorStrength>rColorStrength && rColorStrength>bColorStrength) {
   
   gColorStrength = 255;
   rColorStrength = rColorStrength/2;
   bColorStrength = 0;
 }
 
   if(gColorStrength>bColorStrength && bColorStrength>rColorStrength) {
   
   gColorStrength = 255;
   bColorStrength = bColorStrength/2;
   rColorStrength = 0;
 }
 
   if(bColorStrength>rColorStrength && rColorStrength>gColorStrength) {
   
   bColorStrength = 255;
   rColorStrength = rColorStrength/2;
   gColorStrength = 0;
 }
 
    if(bColorStrength>gColorStrength && gColorStrength>rColorStrength) {
   
   bColorStrength = 255;
   gColorStrength = gColorStrength/2;
   rColorStrength = 0;
 }
 
 
//Now lets measure distance to target from the ultrasonic sensor
  digitalWrite(trigPin, LOW); //Set trigger pin low
  delayMicroseconds(2000); //Let signal settle
  digitalWrite(trigPin, HIGH); //Set trigPin high
  delayMicroseconds(15); //Delay in high state
  digitalWrite(trigPin, LOW); //ping has now been sent
  delayMicroseconds(10); //Delay in low state
  pingTime = pulseIn(echoPin, HIGH);  //pingTime is presented in microceconds
  pingTime=pingTime/1000000; //convert pingTime to seconds by dividing by 1000000 (microseconds in a second)
  pingTime=pingTime/3600; //convert pingtime to hours by dividing by 3600 (seconds in an hour)
  targetDistance= speedOfSound * pingTime;  //This will be in miles, since speed of sound was miles per hour
  targetDistance=targetDistance/2; //Remember ping travels to target and back from target, so you must divide by 2 for actual target distance.
  targetDistance= targetDistance*63360;    //Convert miles to inches by multipling by 63360 (inches per mile)
 
 //Now lets print our data to the serial monitor all on one line divided by commas.
 
  Serial.print(rColorStrength);
  Serial.print(" , ");
  Serial.print(gColorStrength);
  Serial.print(" , ");
  Serial.print(bColorStrength);
  Serial.print(" , ");
  Serial.println(targetDistance);
  
  delay(150);
  
}

In [ ]:
# .py code
## This program reads data over the serial port
## from that arduino. You have to read an entire line of data
## and then you have to parse it into different number values
## Then those R, G, B numbers are used to make the color of
## a visual object in python change.
 
import serial #import serial library
from visual import * #import vPython library
 
MyScene=display(title='My Virtual World') #Create your scene and give it a title.
 
MyScene.width=800  #We can set the dimension of your visual box. 800X800 pixels works well on my screen
MyScene.height= 800
MyScene.autoscale=False #We want to set the range of the scene manually for better control. Turn autoscale off
MyScene.range = (12,12,12) #Set range of your scene to be 12 inches by 12 inches by 12 inches. 
target=box(length=.1, width=10,height=5, pos=(-6,0,0)) #Create the object that will represent your target (which is a colored card for our project)
myBoxEnd=box(length=.1, width=10,height=5, pos=(-8.5,0,0)) #This object is the little square that is the back of the ultrasonic sensor
myTube2=cylinder(color=color.blue, pos=(-8.5,0,-2.5), radius=1.5,length=2.5 ) #One of the 'tubes' in the front of the ultrasonic sensor
myTube3=cylinder(color=color.blue, pos=(-8.5,0,2.5), radius=1.5,length=2.5 )  #Other tube
myBall=sphere(color=color.red, radius=.3)
 
sensorData= serial.Serial('com11',115200) # Create senorData object to read serial port data coming from arduino
 
while True: #This is a while loop that will loop forever, since True is always True.
    
    rate(25) #We need to tell Vpython how fast to go through the loop. 25 times a second works pretty well
    while(sensorData.inWaiting()==0): # Wait here untill there is data on the Serial Port
        pass                          # Do nothing, just loop until data arrives
    textline = sensorData.readline()     # read the entire line of text
    dataNums=textline.split(',')       #Remember to split the line of text into an array at the commas
    red=float(dataNums[0])             # Make variables for Red, Blue, Green. Remember
    green=float(dataNums[1])            # the array was read as text, so must be converted
    blue=float(dataNums[2])           # to numbers with float command
    dist=float(dataNums[3])            # last number in the list is the distance
 
    blue=blue*.7                      #On my sensor, blue is always a little too strong, so I tone it down a little
    if (dist>=1.5 and dist<=2.25):    #only change color or target if target is between 1.5 and 2.25 inches from sensor
        target.color=(red/255., green/255., blue/255.) #This keeps color from flickering.
 
    target.pos=vector(-6 + dist,0,0)  #Adjust the position of the target object to match the distance of the real target from the real sensor

# Lesson 6 : Installing PIP

In [ ]:
Skip

# Lesson 7 : Installing Matplotlib for Graphing and Charting Data

1. Install numpy
2. Install Matplotlib

In [ ]:
# pip install Matplotlib

In [ ]:
# example code
import numpy as np
import matplotlib.pyplot as plt
 
x=[]
y=[]
 
 
for i in np.arange(0,2*np.pi,2*np.pi/1000):
    x.append(i)
    y.append(np.sin(i))

# setting all the plot before plot show
plt.plot(x,y, 'b-', linewidth=2)
plt.grid(True)                  #enable grid
plt.axis([0,2*np.pi,-1.5,1.5])
plt.title('My Sine Wave')       #name the plot
plt.xlabel('Time in Seconds')   #labelling x
plt.ylabel('Sin(t)')            #labelling y

# plot show
plt.show()

# Lesson 8 : Simple Introduction to Matplotlib

In [ ]:
# example

import numpy as np #import numpy library
import matplotlib.pyplot as plt #import matplotlib
 
x= np.linspace( 0, 2*np.pi, 50)#create your x array
y= np.sin(x) #create y array
z= np.cos(x) #create z array
plt.plot(x,y, 'b-d', linewidth=2, label='sinx') #plot y
plt.plot(x,z, 'r-o', linewidth=2, label='cosx') #plot z
plt.grid(True) #display background grid
plt.axis([0,2*np.pi,-1.5,1.5]) #set range on axis
plt.title('My Sin and Cos Waves') #chart title
plt.xlabel('Time in Seconds') #label x axis
plt.ylabel('My Waves') #label y axis
plt.legend() #show legend
plt.show() #show the plot

# Lesson 9 : Measuring Pressure and Temperature with the BMP180 Sensor

Download sensor python library : https://learn.adafruit.com/bmp085/using-the-bmp085

# Lesson 10 : Installing drawnow to Enable Live Graphing in Python with Matplotlib

pip install drawnow

# Lesson 11 : Graphing and Plotting Live Data from Arduino with Matplotlib

In [ ]:
# .ino code

#include "Wire.h"    // imports the wire library for talking over I2C 
#include "Adafruit_BMP085.h"  // import the Pressure Sensor Library
Adafruit_BMP085 mySensor;  // create sensor object called mySensor
 
float tempC;  // Variable for holding temp in C
float tempF;  // Variable for holding temp in F
float pressure; //Variable for holding pressure reading
 
void setup(){
Serial.begin(9600); //turn on serial monitor
mySensor.begin();   //initialize mySensor
}
 
void loop() {
tempC = mySensor.readTemperature(); //  Read Temperature
tempF = tempC*1.8 + 32.; // Convert degrees C to F
pressure=mySensor.readPressure(); //Read Pressure
 
Serial.print("The Temp is: "); //Print Your results
Serial.print(tempF);
Serial.println(" degrees F");
Serial.print("The Pressure is: ");
Serial.print(pressure);
Serial.println(" Pa.");
Serial.println("");
delay(250); //Pause between readings.
}

In [ ]:
# .py code

import serial # import Serial Library
import numpy  # Import numpy
import matplotlib.pyplot as plt #import matplotlib library
from drawnow import *
 
tempF= []
pressure=[]
arduinoData = serial.Serial('com11', 115200) #Creating our serial object named arduinoData
plt.ion() #Tell matplotlib you want interactive mode to plot live data
cnt=0
 
def makeFig(): #Create a function that makes our desired plot
    plt.ylim(80,90)                                 #Set y min and max values
    plt.title('My Live Streaming Sensor Data')      #Plot the title
    plt.grid(True)                                  #Turn the grid on
    plt.ylabel('Temp F')                            #Set ylabels
    plt.plot(tempF, 'ro-', label='Degrees F')       #plot the temperature
    plt.legend(loc='upper left')                    #plot the legend
    plt2=plt.twinx()                                #Create a second y axis
    plt.ylim(93450,93525)                           #Set limits of second y axis- adjust to readings you are getting
    plt2.plot(pressure, 'b^-', label='Pressure (Pa)') #plot pressure data
    plt2.set_ylabel('Pressrue (Pa)')                    #label second y axis
    plt2.ticklabel_format(useOffset=False)           #Force matplotlib to NOT autoscale y axis
    plt2.legend(loc='upper right')                  #plot the legend
    
 
while True: # While loop that loops forever
    while (arduinoData.inWaiting()==0): #Wait here until there is data
        pass #do nothing
    arduinoString = arduinoData.readline() #read the line of text from the serial port
    dataArray = arduinoString.split(',')   #Split it into an array called dataArray
    temp = float( dataArray[0])            #Convert first element to floating number and put in temp
    P =    float( dataArray[1])            #Convert second element to floating number and put in P
    tempF.append(temp)                     #Build our tempF array by appending temp readings
    pressure.append(P)                     #Building our pressure array by appending P readings
    drawnow(makeFig)                       #Call drawnow to update our live graph
    plt.pause(.000001)                     #Pause Briefly. Important to keep drawnow from crashing
    cnt=cnt+1
    if(cnt>50):                            #If you have 50 or more points, delete the first one from the array
        tempF.pop(0)                       #This allows us to just see the last 50 data points
        pressure.pop(0)

# Lesson 12 : Calculating Height from Changes in Pressure

Equation for determine the height from the pressure


<!---![](https://toptechboy.com/wp-content/uploads/2014/07/pressure-height-a.jpg)--->

<!---![](https://toptechboy.com/wp-content/uploads/2014/07/pressure-height-b.jpg)--->


<img src="https://toptechboy.com/wp-content/uploads/2014/07/pressure-height-a.jpg" align="left" width="500" height="500">
<img src="https://toptechboy.com/wp-content/uploads/2014/07/pressure-height-b.jpg" align="right"width="500" height="500">

# Lesson 13 : Calculating Height from BMP180 Pressure Sensor

Using presure sensor to detemine the height    with calibration function 

In [ ]:
# .ino code

#include "Wire.h"    // imports the wire library for talking over I2C 
#include "Adafruit_BMP085.h"  // import the Pressure Sensor Library
Adafruit_BMP085 mySensor;  // create sensor object called mySensor
 
float tempC;  // Variable for holding temp in C
float tempF;  // Variable for holding temp in F
float pressure; //Variable for holding pressure reading
 
void setup(){
Serial.begin(115200); //turn on serial monitor
mySensor.begin();   //initialize mySensor
}
 
void loop() {
tempC = mySensor.readTemperature(); //  Be sure to declare your variables
tempF = tempC*1.8 + 32.; // Convert degrees C to F
pressure=mySensor.readPressure(); //Read Pressure
 
 
Serial.print(tempF);
Serial.print(" , ");
Serial.println(pressure);
delay(250); //Pause between readings.
}

In [ ]:
# .py code

import serial # import Serial Library
import numpy as np  # Import numpy
import matplotlib.pyplot as plt #import matplotlib library
from drawnow import *
import time
 
tempF= []   #Array for our Temperature dat
pressure=[] #Array for our Pressure Data
ht=[]       #Array for our calculated Heights
arduinoData = serial.Serial('com11', 115200) #Creating our serial object named arduinoData
plt.ion() #Tell matplotlib you want interactive mode to plot live data
cnt=0
 
def makeFig(): #Create a function that makes our desired plot
    plt.ylim(0,10)                                 #Set y min and max values
    plt.title('My SuperCool Height-O-Meter')      #Plot the title
    plt.grid(True)                                  #Turn the grid on
    plt.ylabel('Height in Feet')                            #Set ylabels
    plt.plot(ht, 'ro-', label='Height')       #plot the height array
    plt.legend(loc='upper left')                    #plot the legend
    plt2=plt.twinx()                                #Create a second y axis
    plt.ylim(75,85)                           #Set limits of second y axis- adjust to readings you are getting
    plt2.plot(tempF, 'b^-', label='tempF') #plot temperature array
    plt2.set_ylabel('temp F')                    #label second y axis
    plt2.ticklabel_format(useOffset=False)           #Force matplotlib to NOT autoscale y axis
    plt2.legend(loc='upper right')                  #plot the legend
 
tempBucket = 0  #Create bucket to hold sum of temperature readings
PBucket=0       #Create bucket to hold sum of pressure readings
 
print "PLease put Sensor Circuit on Ground for Calibration"  #Calibrate sensor for readings on floor, P0
print "5"           #Give user time to put sensor on floor
time.sleep(1)
print "4"
time.sleep(1)
print "3"
time.sleep(1)
print "2"
time.sleep(1)
print "1"
time.sleep(1)
print "Calibrating Sensor . . ."
 
for i in np.arange(1,11,1):  #Loop ten times to take ten measurements
    while (arduinoData.inWaiting()==0): #Wait here until there is data
        pass #do nothing
    arduinoString = arduinoData.readline() #read the line of text from the serial port
    dataArray = arduinoString.split(',')   #Split it into an array called dataArray
    temp = float( dataArray[0])            #Convert first element to floating number and put in temp
    P =    float( dataArray[1])            #Convert second element to floating number and put in P
    print "P = ",P, " , Temp= ", temp
    tempBucket=tempBucket + temp
    PBucket = PBucket + P
P0 = PBucket/10  #Calcualte average pressure on floor
tempK = ((tempBucket/10)-32)/1.8 +273.15  #Calculate average temperature on floor in K
 
print "Baseline Temp in K is: ", tempK #Print resulrs
print "Baseline Pressure inPa. : ", P0
    
 
 
while True: # While loop that loops forever
    while (arduinoData.inWaiting()==0): #Wait here until there is data
        pass #do nothing
    arduinoString = arduinoData.readline() #read the line of text from the serial port
    dataArray = arduinoString.split(',')   #Split it into an array called dataArray
    temp = float( dataArray[0])            #Convert first element to floating number and put in temp
    P =    float( dataArray[1])            #Convert second element to floating number and put in P
    h=98.57*tempK*np.log(P0/P)
    ht.append(h)
    tempF.append(temp)                     #Build our tempF array by appending temp readings
    pressure.append(P)                     #Building our pressure array by appending P readings
    drawnow(makeFig)                       #Call drawnow to update our live graph
    plt.pause(.000001)                     #Pause Briefly. Important to keep drawnow from crashing
    cnt=cnt+1
    if(cnt>50):                            #If you have 50 or more points, delete the first one from the array
        tempF.pop(0)                       #This allows us to just see the last 50 data points
        pressure.pop(0)
        ht.pop(0)

# Lesson 14 : Introduction to Xbee Radios

Xbee Radion from Sparkfun

# Lesson 15 : Wireless Communication with Xbee Radios

Install XCTU from digi.com
https://www.youtube.com/watch?v=7fRAXAqCaRg

# Lesson 16 : Simple Client Server Model Over Ethernet

Ethernet Shield and Arduino  https://www.youtube.com/watch?v=TWGV47WrKmQ

In [ ]:
# .ino code

#include <Ethernet.h> //Load Ethernet Library
#include <EthernetUdp.h> //Load UDP Library
#include <SPI.h> //Load the SPI Library
 
byte mac[] = { 0xDE, 0xAD, 0xBE, 0xEF, 0xFE, 0xEE}; //Assign a mac address
IPAddress ip(10, 1, 15, 243); //Assign my IP adress
unsigned int localPort = 5000; //Assign a Port to talk over
char packetBuffer[UDP_TX_PACKET_MAX_SIZE];
String datReq; //String for our data
int packetSize; //Size of Packet
EthernetUDP Udp; //Define UDP Object
 
void setup() {
  
Serial.begin(9600); //Turn on Serial Port
Ethernet.begin(mac, ip); //Initialize Ethernet
Udp.begin(localPort); //Initialize Udp
delay(1500); //delay
}
 
void loop() {
  
  packetSize = Udp.parsePacket(); //Read theh packetSize
  
  if(packetSize>0){ //Check to see if a request is present
  
  Udp.read(packetBuffer, UDP_TX_PACKET_MAX_SIZE); //Reading the data request on the Udp
  String datReq(packetBuffer); //Convert packetBuffer array to string datReq
  
  if (datReq =="Red") { //See if Red was requested
  
    Udp.beginPacket(Udp.remoteIP(), Udp.remotePort());  //Initialize Packet send
    Udp.print("You are Asking for Red"); //Send string back to client 
    Udp.endPacket(); //Packet has been sent
  }
   if (datReq =="Green") { //See if Green was requested
  
    Udp.beginPacket(Udp.remoteIP(), Udp.remotePort());  //Initialize Packet send
    Udp.print("You are Asking for Green"); //Send string back to client 
    Udp.endPacket(); //Packet has been sent
   }
    if (datReq =="Blue") { //See if Red was requested
  
    Udp.beginPacket(Udp.remoteIP(), Udp.remotePort());  //Initialize Packet send
    Udp.print("You are Asking for Blue"); //Send string back to client 
    Udp.endPacket(); //Packet has been sent
    }
  }
  memset(packetBuffer, 0, UDP_TX_PACKET_MAX_SIZE);
}

In [ ]:
# .py code
from socket import *
import time
 
address = ( '10.1.15.243', 5000) #Defind who you are talking to (must match arduino IP and port)
client_socket = socket(AF_INET, SOCK_DGRAM) #Set Up the Socket
client_socket.settimeout(1) #only wait 1 second for a resonse
 
while(1): #Main Loop
 
    data = "Blue" #Set data to Blue Command
    client_socket.sendto(data, address) #send command to arduino
    try:
        rec_data, addr = client_socket.recvfrom(2048) #Read response from arduino
        print rec_data #Print the response from Arduino
    except:
        pass
 
    time.sleep(2) #delay before sending next command
 
    data = "Red" #Set data to Blue Command
    client_socket.sendto(data, address) #send command to arduino
    try:
        rec_data, addr = client_socket.recvfrom(2048) #Read response from arduino
        print rec_data #Print the response from Arduino
    except:
        pass
 
    time.sleep(2) #delay before sending next command
    
    data = "Green" #Set data to Blue Command
    client_socket.sendto(data, address) #send command to arduino
    try:
        rec_data, addr = client_socket.recvfrom(2048) #Read response from arduino
        print rec_data #Print the response from Arduino
    except:
        pass
    time.sleep(2) #delay before sending next command

# Lesson 17 : Transferring Data over Ethernet UDP

In [ ]:
# .ino code

#include <Ethernet.h> //Load Ethernet Library
#include <EthernetUdp.h> //Load the Udp Library
#include <SPI.h> //Load SPI Library
 
#include "Wire.h" //imports the wire library
#include "Adafruit_BMP085.h" // import the Pressure/Temperature sensor library
Adafruit_BMP085 mySensor; //Create a sensor object
 
float tempC; //Declare variable for Temp in C
float tempF; //Declare variable for Temp in F
float Pressure; //Declare a variable for Pressure
 
byte mac[] ={ 0xDE, 0xAD, 0xBE, 0xEF, 0xFE, 0xEE}; //Assign mac address
IPAddress ip(10, 1, 15, 243); //Assign the IP Adress
unsigned int localPort = 5000; // Assign a port to talk over
char packetBuffer[UDP_TX_PACKET_MAX_SIZE]; //dimensian a char array to hold our data packet
String datReq; //String for our data
int packetSize; //Size of the packet
EthernetUDP Udp; // Create a UDP Object
 
void setup() {
  
  Serial.begin(9600); //Initialize Serial Port 
  Ethernet.begin( mac, ip); //Inialize the Ethernet
  Udp.begin(localPort); //Initialize Udp
  delay(1500); //delay
  mySensor.begin(); //initialize pressure-temp sensor
  
 
}
 
void loop() {
  
  packetSize =Udp.parsePacket(); //Reads the packet size
  
  if(packetSize>0) { //if packetSize is >0, that means someone has sent a request
    
    Udp.read(packetBuffer, UDP_TX_PACKET_MAX_SIZE); //Read the data request
    String datReq(packetBuffer); //Convert char array packetBuffer into a string called datReq
    
    if (datReq =="Temperature") { //Do the following if Temperature is requested
    
      tempC = mySensor.readTemperature(); //Read the temperature
      tempF = tempC*1.8 + 32; //Convert temp to F
      
      Udp.beginPacket(Udp.remoteIP(), Udp.remotePort()); //Initialize packet send
      Udp.print(tempF); //Send the temperature data
      Udp.endPacket(); //End the packet
      
    }
    
    if (datReq== "Pressure") { //Do the following if Pressure is requested
    
      Pressure=mySensor.readPressure(); //read the pressure
      
        Udp.beginPacket(Udp.remoteIP(), Udp.remotePort()); //Initialize packet send
        Udp.print(Pressure); //Send the Pressure data
        Udp.endPacket(); //End the packet
        
    }
      
  }
  memset(packetBuffer, 0, UDP_TX_PACKET_MAX_SIZE); //clear out the packetBuffer array
}

In [ ]:
# .py code

from socket import *
import time
 
address= ( '10.1.15.243', 5000) #define server IP and port
client_socket =socket(AF_INET, SOCK_DGRAM) #Set up the Socket
client_socket.settimeout(1) #Only wait 1 second for a response
 
while(1):
 
    data = "Temperature" #Set data request to Temperature
 
    client_socket.sendto( data, address) #Send the data request
 
    try:
 
        rec_data, addr = client_socket.recvfrom(2048) #Read response from arduino
        temp = float(rec_data) #Convert string rec_data to float temp
        print "The Measured Temperature is ", temp, " degrees F." # Print the result
 
    except:
        pass
 
    time.sleep(2) #delay before sending next command
 
    data = "Pressure" #Set data request to Pressure
 
    client_socket.sendto( data, address) #Send the data request
 
    try:
 
        rec_data, addr = client_socket.recvfrom(2048) #Read response from arduino
        Pressure = float(rec_data) #Convert string rec_data to float temp
        print "The Measured Pressure is ", Pressure, " Pa." # Print the result
 
    except:
        pass
 
    time.sleep(2) #delay before sending next command
 
    print ""